# Get Figure 10 Data

In [1]:
# run the model validation notebook, which creates a variable `fig10_models`,
# which is a list of models with the necessary fields
experiment_id = 'piControl'
%run find_models.ipynb

models to use for figure 10
GFDL-CM4
GFDL-ESM4


models to reject
models to use for figure 11
GFDL-CM4


models to reject
ACCESS-CM2
ACCESS-ESM1-5
AWI-ESM-1-1-LR
CESM2
CESM2-FV2
CMCC-CM2-SR5
CMCC-ESM2
CNRM-CM6-1
CNRM-CM6-1-HR
CNRM-ESM2-1
CanESM5
EC-Earth3
EC-Earth3-AerChem
EC-Earth3-LR
HadGEM3-GC31-LL
HadGEM3-GC31-MM
IITM-ESM
INM-CM4-8
INM-CM5-0
IPSL-CM6A-LR
MPI-ESM1-2-HR
MPI-ESM1-2-LR
NorESM2-LM
NorESM2-MM
TaiESM1
UKESM1-0-LL


In [2]:
# Domain we wish to study

# test domain 
##################################################################
#lats = (15, 20) # lat min, lat max
#lons = (25, 29) # lon min, lon max
#years = (2013, 2015) # start year, end year
##################################################################

# Thompson, MB
##################################################################
lats = (51, 57) # lat min, lat max
lons = (259, 265) # lon min, lon max
years = (1960, 2015) # start year, end year (note, no leap days)
years = (400, 500)
##################################################################

save_data = True # save as netcdf for further processing?

In [3]:
# remove problem models
#fig10_models.remove('MIROC-ES2L')
#fig10_models.remove('MIROC6')
#fig10_models.remove('SAM0-UNICON')

In [4]:
# parse each model in the list 
for source in fig10_models:
    source_id = source
    
    # get all the 3hr fields
    table_id = '3hr'
    %run CMIP6_lib.ipynb
    required_fields = ['tas', 'huss']
    print(f"""Fetching domain:
          {source_id = }
          {experiment_id = }
          {lats = }
          {lons = }
          {years = }""")
    print("acquiring 3hrly data")
    # grab all fields of interest and combine (3hr)
    my_fields = [get_field(field, df_in) for field in required_fields]
    small_fields = [trim_field(field, lats, lons, years) for field in my_fields]
    ds_3h = xr.combine_by_coords(small_fields, compat="override", combine_attrs="drop_conflicts")
    
    # filter extraneous dimensions
    for dim in ["height", "time_bounds", "depth", "depth_bounds"]:
        try:
            ds_3h = ds_3h.drop(dim)
        except:
            pass
        
    # take spatial average
    #ds_3h = ds_3h.mean(dim=("lat", "lon"))

    print("acquiring daily data")
    # get all the daily fields
    table_id = 'day'
    %run CMIP6_lib.ipynb
    required_fields = ['mrsos']
    
    # grab all fields of interest and combine (day)
    my_fields = [get_field(field, df_in) for field in required_fields]
    small_fields = [trim_field(field, lats, lons, years) for field in my_fields]
    ds_day = xr.combine_by_coords(small_fields, compat="override", combine_attrs="drop_conflicts")
    
    # filter extraneous dimensions
    for dim in ["height", "time_bounds", "depth", "depth_bounds", "lat_bnds", "lon_bnds"]:
        try:
            ds_day = ds_day.drop(dim)
        except:
            pass
    
    # take spatial average
    #ds_day = ds_day.mean(dim=("lat", "lon"))
    
    # interpolate daily data onto the 3hourly and merge.
    print("interpolating")
    day_interp = ds_day.interp_like(ds_3h).chunk({"time":-1})
    print("scrubbing NaN values")
    day_interp = day_interp.interpolate_na(dim="time")
    print("merging datasets")
    full_dataset = ds_3h.merge(day_interp).metpy.quantify().chunk({"time":10000})
    
    
    if save_data:
        print(f"saving {source_id} to disk as netcdf")
        full_dataset.to_netcdf(f"./data/{source_id}-{experiment_id}-fig10.nc", engine="netcdf4")
        
        print("success\n\n")
    else:
        print(f"successfully parsed {source_id}\n\n")

Fetching domain:
          source_id = 'GFDL-CM4'
          experiment_id = 'piControl'
          lats = (51, 57)
          lons = (259, 265)
          years = (400, 500)
acquiring 3hrly data
acquiring daily data
interpolating
scrubbing NaN values
merging datasets
saving GFDL-CM4 to disk as netcdf
success


Fetching domain:
          source_id = 'GFDL-ESM4'
          experiment_id = 'piControl'
          lats = (51, 57)
          lons = (259, 265)
          years = (400, 500)
acquiring 3hrly data
acquiring daily data
interpolating
scrubbing NaN values
merging datasets
saving GFDL-ESM4 to disk as netcdf
success


